# **Importing Necessary libraries**


In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re # regular expression
from keras.layers import SimpleRNN

2024-06-24 04:19:37.120619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 04:19:37.120680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 04:19:37.121770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 04:19:37.128828: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# **Preparing the data named IMDB**

In [2]:
# !pip install nltk

In [3]:
data = pd.read_csv('/workspace/Usrf/Day7/IMDB Dataset (2).csv')
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


Stop Word is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)
Declaring the english stop words

In [4]:
import nltk
nltk.download("stopwords")
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Load and Clean Dataset**
**In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in load_dataset() function, beside loading the dataset using pandas, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.**

# **Encode Sentiments**
**In the same function, We also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.**

In [5]:
def load_dataset():
    df = pd.read_csv('/workspace/Usrf/Day7/IMDB Dataset (2).csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


# **Split Dataset**
**In this work, We decided to split the data into 80% of Training and 20% of Testing set using train_test_split method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.**

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
32906    [this, essentially, variation, house, of, wax,...
43969    [i, saw, hot, millions, tcm, i, completely, fo...
16045    [ok, first, said, i, wanted, check, whether, m...
30207    [i, saw, version, decade, ago, looking, ever, ...
26341    [gordon, goes, top, typical, full, moon, fashi...
                               ...                        
26429    [i, believe, show, still, rating, i, could, se...
15083    [yes, movie, hilarious, acting, top, notch, wh...
5917     [i, avid, fan, violent, exploitation, cinema, ...
30662    [make, wonder, first, place, this, movie, bad,...
2063     [but, i, doubt, many, running, see, movie, or,...
Name: review, Length: 40000, dtype: object 

48639    [when, i, kid, always, used, babysat, always, ...
38906    [a, brilliant, movie, family, guilt, sacrifice...
17539    [this, movie, terrible, throughout, whole, mov...
48837    [i, enjoyed, carax, les, amants, du, pont, neu...
23205    [stanley, iris, heart, warming, film, two, peo...
 

**Function for getting the average review length, by calculating the mean of all the reviews length (using numpy.mean)**

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

# **Tokenize and Pad/Truncate Reviews**
**A Neural Network only accepts numeric data, so we need to encode the reviews. I use tensorflow.keras.preprocessing.text.Tokenizer to encode the reviews into integers, where each unique word is automatically indexed (using fit_on_texts method) based on x_train.**

**x_train and x_test is converted into integers using texts_to_sequences method.**

**Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.**

In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding
print('Total Words:', total_words)

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Total Words: 92597
Encoded X Train
 [[    8  1852  6826 ...   356   259   516]
 [    1   121   777 ...     0     0     0]
 [  485    23   208 ... 14648  2894  1424]
 ...
 [    1  6302   233 ...   232  6876   134]
 [   25   486    23 ...     0     0     0]
 [   30     1   693 ...   150  3349  3349]] 

Encoded X Test
 [[  173     1   444 ...     0     0     0]
 [   39   419     3 ...     0     0     0]
 [    8     3   286 ...     0     0     0]
 ...
 [    8    21   310 ...     0     0     0]
 [  424 45002 40587 ...     0     0     0]
 [ 1135  7095  1736 ...   257  1254  1328]] 

Maximum review length:  130


# **Build Architecture/Model**
**Embedding Layer: in simple terms, it creates word vectors of each word in the word_index and group words that are related or have similar meaning by analyzing other words around them.**

**RNN Layer: to make a decision to keep or throw away data by considering the current input, previous output.**

**Dense Layer: compute the input with the weight matrix and bias (optional), and using an activation function. I use Sigmoid activation function for this work because the output is only 0 or 1.**

**The optimizer is Adam and the loss function is Binary Crossentropy because again the output is only 0 and 1, which is a binary number.**

In [9]:
rnn = Sequential()

rnn.add(Embedding(total_words,32,input_length =max_length))
rnn.add(SimpleRNN(64,input_shape = (total_words, max_length), return_sequences=False,activation="relu"))
rnn.add(Dense(1, activation = 'sigmoid')) #flatten

print(rnn.summary())
rnn.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2963104   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2969377 (11.33 MB)
Trainable params: 2969377 (11.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


2024-06-24 04:19:48.446850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17947 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:b7:00.0, compute capability: 8.0


#**Trainin the Model**

In [10]:
history = rnn.fit(x_train,y_train,epochs = 20,batch_size=128,verbose = 1)

Epoch 1/20


2024-06-24 04:19:50.171122: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4e3e3ba240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-24 04:19:50.171182: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 3g.20gb, Compute Capability 8.0
2024-06-24 04:19:50.177000: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-24 04:19:50.213467: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
I0000 00:00:1719202790.305639  519961 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 [==============================] - 59s 183ms/step - loss: 0.6942 - accuracy: 0.5079
Epoch 2/20
313/313 [==============================] - 40s 129ms/step - loss: 0.7098 - accuracy: 0.5480
Epoch 3/20
313/313 [==============================] - 31s 100ms/step - loss: 0.6572 - accuracy: 0.6007
Epoch 4/20
313/313 [==============================] - 28s 89ms/step - loss: 0.5317 - accuracy: 0.7504
Epoch 5/20
313/313 [==============================] - 26s 84ms/step - loss: 0.6123 - accuracy: 0.6037
Epoch 6/20
313/313 [==============================] - 25s 80ms/step - loss: 0.5757 - accuracy: 0.6673
Epoch 7/20
313/313 [==============================] - 24s 78ms/step - loss: 0.5642 - accuracy: 0.6904
Epoch 8/20
313/313 [==============================] - 24s 76ms/step - loss: 0.4223 - accuracy: 0.8372
Epoch 9/20
313/313 [==============================] - 23s 74ms/step - loss: 0.3467 - accuracy: 0.8749
Epoch 10/20
313/313 [==============================] - 22s 71ms/step - loss: 0.6464 - accu

# **Saving The Model**

In [11]:
model = rnn.save('rnn.h5')
loaded_model = load_model('rnn.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#**Evaluation**

In [12]:
y_pred = rnn.predict(x_test, batch_size = 128)
print(y_pred)
print(y_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i] = 1
  else:
    y_pred[i] = 0

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 [==============================] - 1s 10ms/step
[[0.48266408]
 [0.48266298]
 [0.48266408]
 ...
 [0.4826619 ]
 [0.4826619 ]
 [0.56367934]]
48639    0
38906    1
17539    0
48837    0
23205    1
        ..
37415    0
13480    0
1811     1
32468    1
13481    1
Name: sentiment, Length: 10000, dtype: int64
Correct Prediction: 5651
Wrong Prediction: 4349
Accuracy: 56.510000000000005


Message: **Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!**

# **Example review**

In [ ]:
review = str(input('Movie Review: '))

#**Pre-processing of entered review**

In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]') #Compile a regular expression to remove unwanted characters
review = regex.sub('', review) #Remove unwanted characters from the review
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops] #Filter out stop words
filtered = ' '.join(filtered) #Join the filtered words back into a string
filtered = [filtered.lower()] #Convert the filtered string to lowercase

print('Filtered: ', filtered)

In [ ]:
tokenize_words = token.texts_to_sequences(filtered) #Tokenize the filtered text
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

#**Prediction**

In [ ]:
result = rnn.predict(tokenize_words)
print(result)

In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')